# 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)

# Latent Dirichlet Allocation (LDA)

- LDA(Latent Dirichlet Allocation)는 토픽 모델링 알고리즘입니다.
- LDA는 각 문서의 토픽 분포와 각 토픽 내의 단어 분포를 추정합니다.


- 사용자는 LDA에 몇 개의 토픽을 찾을지 결정하며, 이를 하이퍼파라미터 K 로 설정 합니다.


- 예를 들어, K=2(num_topics) 일 때, 알고리즘이 두 개의 토픽을 찾도록 요청합니다.
- 예를 들어, 두 개의 토픽을 찾았을 때, 각 문서는 두 토픽 중 하나에 속하며, 그 분포를 표시합니다.


- 또한 각 토픽은 특정 단어들의 분포를 보여줍니다.
- 이를 통해 사용자는 문서의 주제를 파악할 수 있습니다.

- LDA(Latent Dirichlet Allocation)는 문서의 집합에서 토픽을 찾아내는 알고리즘입니다. 
- 이를 위해 LDA는 문서의 단어들을 빈도수 기반의 표현 방법인 Bag of Words(BOW)나 TF-IDF 행렬을 입력으로 사용합니다. 


- LDA는 단어의 순서는 고려하지 않고, 각 문서가 작성될 때 사용된 토픽과 단어를 역추적하여 토픽을 추출합니다. 
- 이를 통해 문서의 주제를 파악할 수 있습니다.


- 각 문서는 단어의 혼합을 특정 주제로 정하고, 해당 주제에서 단어를 선택하여 작성된 것으로 가정됩니다.
- LDA는 이러한 가정을 바탕으로 역공학을 수행하여 문서로부터 토픽을 추출합니다.

[프로세스]
1. 문서에 사용할 단어의 개수 K(num_topics)을 정한다.
2. 문서에 사용할 토픽의 혼합을 확률 분포에 기반하여 결정한다.
3. 문서에 사용할 각 단어를 정한다.
   1) 토픽 분포에서 토픽 T를 확률적으로 고른다.
   2) 선택한 토픽 T 에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어를 고른다.

In [1]:
# 부모 폴더의 경로 추가
import sys; sys.path.insert(0, '..')

from util.data_loader import DataLoader
from util.metric_calculator import MetricCalculator

In [2]:
# Movielens 데이터 로딩
data_loader = DataLoader(num_users=1000, num_test_items=5, data_path='../data/ml-10M100K/')
movielens = data_loader.load()

In [3]:
import gensim
import logging
from gensim.corpora.dictionary import Dictionary

movie_content = movielens.item_content.copy()
# tag가 부여되어 있지 않은 영화도 있지만, genre는 모든 영화에 부여되어 있다
# tag와 genre를 결합한 것을 영화 콘텐츠 정보로 해서 비슷한 영화를 찾아 추천한다
# tag가 없는 영화는 NaN으로 되어 있으므로, 빈 리스트로 변환한 뒤 처리한다
movie_content['tag_genre'] = movie_content['tag'].fillna("").apply(list) + movie_content['genre'].apply(list)
movie_content['tag_genre'] = movie_content['tag_genre'].apply(lambda x:list(map(str, x)))

# 태그와 장르 데이터를 사용해 LDA를 학습한다
tag_genre_data = movie_content.tag_genre.tolist()

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
common_dictionary = Dictionary(tag_genre_data)
common_corpus = [common_dictionary.doc2bow(text) for text in tag_genre_data]

# LDA 학습
lda_model = gensim.models.LdaModel(common_corpus, id2word=common_dictionary, num_topics=50, passes=30)
lda_topics = lda_model[common_corpus]

C:\Users\jkm20\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
2024-03-31 21:54:37,557 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-03-31 21:54:37,809 : INFO : adding document #10000 to Dictionary<14749 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...>
2024-03-31 21:54:37,823 : INFO : built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)
2024-03-31 21:54:37,825 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)", 'datetime': '2024-03-31T21:54:37.824976', 'gensim': '4.3.2', 'python': '3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'crea

2024-03-31 21:54:40,573 : INFO : topic #43 (0.020): 0.044*"marx brothers" + 0.039*"bill murray" + 0.033*"memory" + 0.027*"Comedy" + 0.027*"jack nicholson" + 0.025*"jane austen" + 0.023*"Drama" + 0.021*"wedding" + 0.018*"liam neeson" + 0.018*"hugh grant"
2024-03-31 21:54:40,573 : INFO : topic #2 (0.020): 0.119*"Drama" + 0.080*"War" + 0.050*"criterion" + 0.043*"Action" + 0.033*"martial arts" + 0.026*"Comedy" + 0.026*"Adventure" + 0.018*"poignant" + 0.018*"deliberate" + 0.017*"reflective"
2024-03-31 21:54:40,579 : INFO : topic diff=0.231356, rho=0.447214
2024-03-31 21:54:40,845 : INFO : -21.396 per-word bound, 2759379.7 perplexity estimate based on a held-out corpus of 681 documents with 5199 words
2024-03-31 21:54:40,845 : INFO : PROGRESS: pass 0, at document #10681/10681
2024-03-31 21:54:40,999 : INFO : merging changes from 681 documents into a model of 10681 documents
2024-03-31 21:54:41,029 : INFO : topic #9 (0.020): 0.198*"Thriller" + 0.137*"Mystery" + 0.105*"Drama" + 0.056*"r" + 0.0

2024-03-31 21:54:42,371 : INFO : PROGRESS: pass 1, at document #10000/10681
2024-03-31 21:54:42,623 : INFO : merging changes from 2000 documents into a model of 10681 documents
2024-03-31 21:54:42,668 : INFO : topic #32 (0.020): 0.036*"underrated" + 0.025*"michael crichton" + 0.022*"janus 50" + 0.022*"1980s" + 0.017*"business" + 0.017*"brian de palma" + 0.015*"canada" + 0.015*"mother-son relationship" + 0.013*"roald dahl" + 0.012*"Drama"
2024-03-31 21:54:42,668 : INFO : topic #18 (0.020): 0.190*"anime" + 0.091*"japan" + 0.037*"pg" + 0.023*"Animation" + 0.022*"movie to see" + 0.016*"affectionate" + 0.015*"Drama" + 0.015*"fashion" + 0.013*"president" + 0.012*"romance"
2024-03-31 21:54:42,668 : INFO : topic #31 (0.020): 0.296*"Comedy" + 0.194*"Documentary" + 0.075*"zombies" + 0.057*"documentary" + 0.016*"zombie" + 0.013*"to see" + 0.012*"humorous" + 0.011*"campy" + 0.011*"light" + 0.010*"sundance award winner"
2024-03-31 21:54:42,668 : INFO : topic #10 (0.020): 0.032*"Drama" + 0.030*"keir

2024-03-31 21:54:44,298 : INFO : topic #39 (0.020): 0.059*"mel gibson" + 0.058*"samuel l. jackson" + 0.051*"uma thurman" + 0.041*"kidnapping" + 0.029*"steve martin" + 0.027*"revenge" + 0.019*"predictable" + 0.019*"witch" + 0.014*"hollywood" + 0.013*"bfi classic"
2024-03-31 21:54:44,298 : INFO : topic #43 (0.020): 0.051*"jack nicholson" + 0.051*"memory" + 0.042*"bill murray" + 0.033*"marx brothers" + 0.030*"Comedy" + 0.029*"british" + 0.028*"wedding" + 0.026*"boring" + 0.026*"jane austen" + 0.019*"hugh grant"
2024-03-31 21:54:44,298 : INFO : topic #4 (0.020): 0.068*"surreal" + 0.064*"quirky" + 0.046*"satirical" + 0.034*"irreverent" + 0.031*"dark comedy" + 0.030*"humorous" + 0.030*"biting" + 0.030*"cynical" + 0.030*"nicolas cage" + 0.028*"black comedy"
2024-03-31 21:54:44,298 : INFO : topic #24 (0.020): 0.110*"romance" + 0.049*"chick flick" + 0.038*"pg-13" + 0.037*"nonlinear" + 0.033*"girlie movie" + 0.032*"video game adaptation" + 0.028*"Comedy" + 0.027*"bechdel test:fail" + 0.023*"juli

2024-03-31 21:54:46,068 : INFO : topic #11 (0.020): 0.133*"comedy" + 0.069*"Comedy" + 0.060*"funny" + 0.028*"hilarious" + 0.028*"seen more than once" + 0.022*"jim carrey" + 0.020*"parody" + 0.017*"stupid" + 0.017*"quirky" + 0.016*"funny as hell"
2024-03-31 21:54:46,068 : INFO : topic diff=0.110822, rho=0.327201
2024-03-31 21:54:46,078 : INFO : PROGRESS: pass 3, at document #8000/10681
2024-03-31 21:54:46,543 : INFO : merging changes from 2000 documents into a model of 10681 documents
2024-03-31 21:54:46,588 : INFO : topic #32 (0.020): 0.040*"michael crichton" + 0.038*"underrated" + 0.032*"janus 50" + 0.027*"1980s" + 0.022*"brian de palma" + 0.019*"mother-son relationship" + 0.017*"canada" + 0.015*"switching places" + 0.015*"gangster" + 0.014*"russia"
2024-03-31 21:54:46,588 : INFO : topic #9 (0.020): 0.242*"Thriller" + 0.149*"Mystery" + 0.140*"Drama" + 0.053*"nudity (topless - notable)" + 0.046*"Crime" + 0.038*"directorial debut" + 0.028*"r" + 0.023*"Action" + 0.020*"baseball" + 0.019*

...

2024-03-31 21:55:39,876 : INFO : topic #41 (0.020): 0.111*"sports" + 0.048*"peter sellers" + 0.044*"french" + 0.040*"hulu" + 0.037*"africa" + 0.033*"basketball" + 0.029*"gangs" + 0.026*"tragedy" + 0.023*"milla jovovich" + 0.018*"science"
2024-03-31 21:55:39,876 : INFO : topic #7 (0.020): 0.072*"cars" + 0.034*"nazis" + 0.029*"adventure" + 0.028*"archaeology" + 0.026*"mst3k" + 0.026*"john cusack" + 0.023*"want to see again" + 0.021*"fighting" + 0.020*"race" + 0.018*"afternoon section"
2024-03-31 21:55:39,885 : INFO : topic diff=0.039675, rho=0.178627
2024-03-31 21:55:39,885 : INFO : PROGRESS: pass 25, at document #10000/10681
2024-03-31 21:55:40,106 : INFO : merging changes from 2000 documents into a model of 10681 documents
2024-03-31 21:55:40,149 : INFO : topic #30 (0.020): 0.128*"Children" + 0.115*"Musical" + 0.099*"Comedy" + 0.066*"Animation" + 0.064*"disney" + 0.056*"Fantasy" + 0.055*"Adventure" + 0.050*"animation" + 0.036*"musical" + 0.021*"children"
2024-03-31 21:55:40,149 : INFO 

2024-03-31 21:55:41,438 : INFO : topic diff=0.044364, rho=0.175844
2024-03-31 21:55:41,438 : INFO : PROGRESS: pass 26, at document #8000/10681
2024-03-31 21:55:41,816 : INFO : merging changes from 2000 documents into a model of 10681 documents
2024-03-31 21:55:41,857 : INFO : topic #28 (0.020): 0.070*"Drama" + 0.050*"true story" + 0.048*"drama" + 0.036*"history" + 0.036*"biography" + 0.032*"based on a true story" + 0.029*"music" + 0.029*"holocaust" + 0.027*"edward norton" + 0.019*"boxing"
2024-03-31 21:55:41,859 : INFO : topic #40 (0.020): 0.326*"less than 300 ratings" + 0.168*"Drama" + 0.065*"prison" + 0.032*"sven's to see list" + 0.028*"friendship" + 0.024*"War" + 0.014*"adaptation" + 0.009*"united states" + 0.008*"women's lib" + 0.008*"trilogy"
2024-03-31 21:55:41,860 : INFO : topic #36 (0.020): 0.097*"magic" + 0.060*"sequel" + 0.051*"harrison ford" + 0.044*"fantasy" + 0.029*"george lucas" + 0.028*"funniest movies" + 0.022*"brazil" + 0.021*"space opera" + 0.020*"adventure" + 0.015*"

2024-03-31 21:55:43,735 : INFO : topic #4 (0.020): 0.075*"quirky" + 0.062*"surreal" + 0.054*"humorous" + 0.048*"dark comedy" + 0.039*"satirical" + 0.032*"black comedy" + 0.031*"Comedy" + 0.030*"nicolas cage" + 0.030*"irreverent" + 0.026*"biting"
2024-03-31 21:55:43,737 : INFO : topic #47 (0.020): 0.137*"world war ii" + 0.108*"comic book" + 0.085*"superhero" + 0.051*"war" + 0.046*"War" + 0.045*"super-hero" + 0.027*"adapted from:comic" + 0.022*"Action" + 0.021*"sean penn" + 0.020*"wwii"
2024-03-31 21:55:43,738 : INFO : topic #2 (0.020): 0.125*"Drama" + 0.103*"War" + 0.062*"criterion" + 0.036*"martial arts" + 0.032*"atmospheric" + 0.025*"bleak" + 0.023*"poignant" + 0.021*"tumey's dvds" + 0.019*"jackie chan" + 0.019*"talky"
2024-03-31 21:55:43,738 : INFO : topic #42 (0.020): 0.137*"drugs" + 0.070*"religion" + 0.053*"Drama" + 0.033*"coming of age" + 0.032*"london" + 0.025*"christianity" + 0.022*"depressing" + 0.021*"television" + 0.020*"ewan mcgregor" + 0.019*"notable soundtrack"
2024-03-31

2024-03-31 21:55:45,317 : INFO : topic #31 (0.020): 0.486*"Comedy" + 0.166*"Documentary" + 0.055*"to see" + 0.046*"zombies" + 0.030*"documentary" + 0.009*"zombie" + 0.008*"sam raimi" + 0.007*"sundance award winner" + 0.006*"bechdel test:pass" + 0.005*"upbeat"
2024-03-31 21:55:45,317 : INFO : topic #13 (0.020): 0.239*"nudity (topless)" + 0.159*"nudity (topless - brief)" + 0.056*"Drama" + 0.053*"teen" + 0.051*"nudity (rear)" + 0.030*"will smith" + 0.022*"virtual reality" + 0.021*"food" + 0.017*"meg ryan" + 0.013*"john wayne"
2024-03-31 21:55:45,317 : INFO : topic diff=0.026753, rho=0.170646
2024-03-31 21:55:45,317 : INFO : PROGRESS: pass 28, at document #6000/10681
2024-03-31 21:55:45,515 : INFO : merging changes from 2000 documents into a model of 10681 documents
2024-03-31 21:55:45,549 : INFO : topic #38 (0.020): 0.178*"fantasy" + 0.073*"racism" + 0.041*"russell crowe" + 0.030*"courtroom" + 0.027*"courtroom drama" + 0.018*"court" + 0.017*"adventure" + 0.015*"stage" + 0.014*"judaism" + 

2024-03-31 21:55:46,733 : INFO : topic diff=0.121515, rho=0.168215
2024-03-31 21:55:46,735 : INFO : PROGRESS: pass 29, at document #4000/10681
2024-03-31 21:55:47,065 : INFO : merging changes from 2000 documents into a model of 10681 documents
2024-03-31 21:55:47,112 : INFO : topic #0 (0.020): 0.052*"Drama" + 0.041*"lyrical" + 0.028*"dark" + 0.026*"enigmatic" + 0.026*"dreamlike" + 0.024*"bittersweet" + 0.022*"beautiful" + 0.022*"whimsical" + 0.022*"hw drama" + 0.021*"meditative"
2024-03-31 21:55:47,112 : INFO : topic #8 (0.020): 0.377*"Drama" + 0.243*"Romance" + 0.179*"Comedy" + 0.027*"pixar" + 0.019*"bibliothek" + 0.014*"movie to see" + 0.010*"Fantasy" + 0.008*"journalism" + 0.008*"library" + 0.007*"football"
2024-03-31 21:55:47,112 : INFO : topic #47 (0.020): 0.130*"world war ii" + 0.112*"comic book" + 0.095*"superhero" + 0.046*"super-hero" + 0.046*"war" + 0.044*"War" + 0.029*"adapted from:comic" + 0.022*"batman" + 0.022*"Action" + 0.021*"sean penn"
2024-03-31 21:55:47,116 : INFO : t

In [4]:
# LDAContent추천
from src.lda_content import LDAContentRecommender
recommender = LDAContentRecommender()
recommend_result = recommender.recommend(movielens)

2024-03-31 21:55:48,475 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-03-31 21:55:48,575 : INFO : adding document #10000 to Dictionary<14749 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...>
2024-03-31 21:55:48,588 : INFO : built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)
2024-03-31 21:55:48,589 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<15261 unique tokens: ['3d', 'Adventure', 'Animation', 'Children', 'Comedy']...> from 10681 documents (total 117144 corpus positions)", 'datetime': '2024-03-31T21:55:48.589759', 'gensim': '4.3.2', 'python': '3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}
2024-03-31 21:55:48,655 : INFO : using symmetric alpha at 0.02
2024-03-31 21:55:48,660 : INFO : using symmetric eta at 0.02
2024-03-31 21:55:48,660 : INFO : 

2024-03-31 21:55:50,674 : INFO : topic #16 (0.020): 0.080*"nudity (rear)" + 0.053*"pg" + 0.041*"military" + 0.035*"claymation" + 0.031*"weird" + 0.030*"nudity (topless)" + 0.024*"aardman" + 0.019*"nudity (topless - brief)" + 0.019*"childhood" + 0.018*"penguins"
2024-03-31 21:55:50,677 : INFO : topic diff=0.241935, rho=0.447214
2024-03-31 21:55:50,825 : INFO : -21.496 per-word bound, 2957512.5 perplexity estimate based on a held-out corpus of 681 documents with 5199 words
2024-03-31 21:55:50,825 : INFO : PROGRESS: pass 0, at document #10681/10681
2024-03-31 21:55:50,940 : INFO : merging changes from 681 documents into a model of 10681 documents
2024-03-31 21:55:51,003 : INFO : topic #36 (0.020): 0.035*"russell crowe" + 0.031*"virus" + 0.029*"not available from netflix" + 0.028*"Comedy" + 0.026*"based on a book" + 0.025*"cult film" + 0.025*"peter jackson" + 0.024*"movielens quickpick" + 0.023*"1980s" + 0.021*"adam sandler"
2024-03-31 21:55:51,005 : INFO : topic #6 (0.020): 0.125*"Musical

2024-03-31 21:55:52,625 : INFO : topic #42 (0.020): 0.086*"martial arts" + 0.059*"robin williams" + 0.042*"jonossa" + 0.041*"seen 2006" + 0.031*"japan" + 0.029*"samurai" + 0.027*"Drama" + 0.025*"orson welles" + 0.024*"hugh grant" + 0.023*"divorce"
2024-03-31 21:55:52,625 : INFO : topic #41 (0.020): 0.141*"remake" + 0.058*"new york city" + 0.049*"christmas" + 0.044*"family" + 0.037*"new york" + 0.031*"Drama" + 0.027*"seen" + 0.024*"kevin spacey" + 0.023*"predictable" + 0.021*"friday night movie"
2024-03-31 21:55:52,625 : INFO : topic #5 (0.020): 0.162*"classic" + 0.082*"musical" + 0.027*"afi 100 (laughs)" + 0.024*"los angeles" + 0.018*"witch" + 0.018*"national film registry" + 0.016*"john travolta" + 0.015*"Musical" + 0.014*"adapted from b'way" + 0.013*"movie to see"
2024-03-31 21:55:52,625 : INFO : topic #49 (0.020): 0.081*"national film registry" + 0.026*"natalie portman" + 0.025*"tumey's dvds" + 0.022*"Film-Noir" + 0.020*"film noir" + 0.020*"black and white" + 0.019*"imdb top 250" + 

2024-03-31 21:55:54,666 : INFO : topic #45 (0.020): 0.235*"less than 300 ratings" + 0.109*"Drama" + 0.061*"nudity (topless - notable)" + 0.036*"Comedy" + 0.035*"tim burton" + 0.023*"library" + 0.018*"michael moore" + 0.015*"blindfold" + 0.014*"depressing" + 0.011*"golden raspberry (worst actor)"
2024-03-31 21:55:54,670 : INFO : topic #37 (0.020): 0.185*"War" + 0.109*"world war ii" + 0.103*"Drama" + 0.047*"history" + 0.047*"war" + 0.035*"jim carrey" + 0.025*"Action" + 0.019*"70mm" + 0.014*"nazis" + 0.013*"mental illness"
2024-03-31 21:55:54,670 : INFO : topic #38 (0.020): 0.142*"drugs" + 0.037*"samuel l. jackson" + 0.035*"peter sellers" + 0.030*"ewan mcgregor" + 0.024*"divx" + 0.022*"poverty" + 0.020*"addiction" + 0.019*"hulu" + 0.018*"michael caine" + 0.017*"Drama"
2024-03-31 21:55:54,675 : INFO : topic diff=0.119826, rho=0.346261
2024-03-31 21:55:54,675 : INFO : PROGRESS: pass 2, at document #10000/10681
2024-03-31 21:55:54,982 : INFO : merging changes from 2000 documents into a model

2024-03-31 21:55:56,650 : INFO : topic #11 (0.020): 0.127*"religion" + 0.115*"keanu reeves" + 0.020*"dark" + 0.018*"reality tv" + 0.018*"courtesan" + 0.018*"slash" + 0.017*"slavery" + 0.017*"facebook rec" + 0.014*"class issues" + 0.012*"product placement"
2024-03-31 21:55:56,655 : INFO : topic diff=0.113857, rho=0.327201
2024-03-31 21:55:56,663 : INFO : PROGRESS: pass 3, at document #8000/10681
2024-03-31 21:55:56,885 : INFO : merging changes from 2000 documents into a model of 10681 documents
2024-03-31 21:55:56,955 : INFO : topic #20 (0.020): 0.086*"biography" + 0.060*"Drama" + 0.043*"biopic" + 0.042*"ingmar bergman" + 0.035*"ensemble cast" + 0.031*"julia roberts" + 0.030*"suicide" + 0.028*"corvallis library" + 0.027*"multiple storylines" + 0.025*"medieval"
2024-03-31 21:55:56,964 : INFO : topic #47 (0.020): 0.059*"serial killer" + 0.046*"psychology" + 0.045*"brad pitt" + 0.041*"edward norton" + 0.033*"heist" + 0.031*"hayao miyazaki" + 0.031*"steven spielberg" + 0.029*"crime" + 0.029

...

2024-03-31 21:56:40,343 : INFO : topic #18 (0.020): 0.082*"magic" + 0.079*"gay" + 0.062*"racism" + 0.044*"pg13" + 0.040*"Drama" + 0.027*"food" + 0.027*"social commentary" + 0.023*"homosexuality" + 0.020*"19th century" + 0.019*"denzel washington"
2024-03-31 21:56:40,343 : INFO : topic #19 (0.020): 0.062*"nudity (full frontal)" + 0.061*"drama" + 0.035*"Drama" + 0.034*"philip k. dick" + 0.028*"vietnam war" + 0.027*"rape" + 0.025*"vietnam" + 0.022*"very good" + 0.020*"to see" + 0.016*"slow"
2024-03-31 21:56:40,353 : INFO : topic diff=0.046241, rho=0.175844
2024-03-31 21:56:40,353 : INFO : PROGRESS: pass 26, at document #8000/10681
2024-03-31 21:56:40,613 : INFO : merging changes from 2000 documents into a model of 10681 documents
2024-03-31 21:56:40,678 : INFO : topic #2 (0.020): 0.287*"Documentary" + 0.104*"to see" + 0.064*"documentary" + 0.026*"mockumentary" + 0.025*"sexuality" + 0.023*"star trek" + 0.021*"dogs" + 0.021*"kevin smith" + 0.018*"movie to see" + 0.017*"in netflix queue"
2024

2024-03-31 21:56:41,963 : INFO : topic diff=0.023122, rho=0.173186
2024-03-31 21:56:41,966 : INFO : PROGRESS: pass 27, at document #6000/10681
2024-03-31 21:56:42,143 : INFO : merging changes from 2000 documents into a model of 10681 documents
2024-03-31 21:56:42,182 : INFO : topic #18 (0.020): 0.081*"magic" + 0.079*"gay" + 0.062*"racism" + 0.044*"pg13" + 0.040*"Drama" + 0.027*"food" + 0.027*"social commentary" + 0.023*"homosexuality" + 0.020*"19th century" + 0.019*"denzel washington"
2024-03-31 21:56:42,182 : INFO : topic #32 (0.020): 0.135*"johnny depp" + 0.067*"clint eastwood" + 0.059*"vhs" + 0.045*"jackie chan" + 0.037*"western" + 0.031*"kung fu" + 0.031*"leonardo dicaprio" + 0.030*"spaghetti western" + 0.029*"propaganda" + 0.029*"david lynch"
2024-03-31 21:56:42,182 : INFO : topic #26 (0.020): 0.042*"Drama" + 0.035*"james bond" + 0.024*"007" + 0.023*"bond" + 0.021*"mel gibson" + 0.021*"reflective" + 0.020*"atmospheric" + 0.019*"poignant" + 0.018*"lyrical" + 0.017*"bittersweet"
202

2024-03-31 21:56:43,533 : INFO : topic #7 (0.020): 0.113*"romance" + 0.065*"Romance" + 0.054*"chick flick" + 0.038*"boring" + 0.030*"girlie movie" + 0.027*"baseball" + 0.023*"Comedy" + 0.020*"love story" + 0.019*"whimsical" + 0.017*"wedding"
2024-03-31 21:56:43,534 : INFO : topic #5 (0.020): 0.292*"classic" + 0.062*"musical" + 0.045*"afi 100 (laughs)" + 0.034*"Musical" + 0.030*"national film registry" + 0.021*"afi 100" + 0.016*"70mm" + 0.015*"adapted from b'way" + 0.015*"john travolta" + 0.015*"breakthroughs"
2024-03-31 21:56:43,535 : INFO : topic #31 (0.020): 0.127*"zombies" + 0.074*"horror" + 0.072*"pg-13" + 0.039*"cult classic" + 0.025*"infidelity" + 0.025*"campy" + 0.024*"zombie" + 0.024*"joaquin phoenix" + 0.024*"sam raimi" + 0.020*"books"
2024-03-31 21:56:43,539 : INFO : topic diff=0.022572, rho=0.170646
2024-03-31 21:56:43,541 : INFO : PROGRESS: pass 28, at document #6000/10681
2024-03-31 21:56:43,745 : INFO : merging changes from 2000 documents into a model of 10681 documents
2

2024-03-31 21:56:44,823 : INFO : topic #17 (0.020): 0.330*"Sci-Fi" + 0.071*"Horror" + 0.068*"Action" + 0.044*"video game adaptation" + 0.027*"animals" + 0.025*"movie to see" + 0.022*"football" + 0.018*"futuristic" + 0.015*"milla jovovich" + 0.013*"g"
2024-03-31 21:56:44,823 : INFO : topic #34 (0.020): 0.074*"ghosts" + 0.051*"bill murray" + 0.044*"television" + 0.030*"courtroom" + 0.030*"courtroom drama" + 0.026*"secret service" + 0.021*"roman polanski" + 0.017*"18th century" + 0.017*"opera" + 0.017*"court"
2024-03-31 21:56:44,831 : INFO : topic diff=0.124178, rho=0.168215
2024-03-31 21:56:44,832 : INFO : PROGRESS: pass 29, at document #4000/10681
2024-03-31 21:56:45,023 : INFO : merging changes from 2000 documents into a model of 10681 documents
2024-03-31 21:56:45,053 : INFO : topic #31 (0.020): 0.127*"zombies" + 0.074*"horror" + 0.072*"pg-13" + 0.039*"cult classic" + 0.025*"infidelity" + 0.025*"campy" + 0.024*"zombie" + 0.024*"joaquin phoenix" + 0.024*"sam raimi" + 0.020*"books"
2024

2024-03-31 21:56:46,215 : INFO : LdaModel lifecycle event {'msg': 'trained LdaModel<num_terms=15261, num_topics=50, decay=0.5, chunksize=2000> in 57.50s', 'datetime': '2024-03-31T21:56:46.215775', 'gensim': '4.3.2', 'python': '3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.22621-SP0', 'event': 'created'}


In [5]:
# 평가
metric_calculator = MetricCalculator()
metrics = metric_calculator.calc(
    movielens.test.rating.tolist(), recommend_result.rating.tolist(),
    movielens.test_user2items, recommend_result.user2items, k=10)
print(metrics)

rmse=0.000, Precision@K=0.004, Recall@K=0.012


#  딥 러닝을 이용한 자연어 처리 입문
## 19. 토픽 모델링(Topic Modeling) 
### 19-03 사이킷런의 잠재 디리클레 할당

In [8]:
import pandas as pd
import urllib.request
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/19.%20Topic%20Modeling%20(LDA%2C%20BERT-Based)/dataset/abcnews-date-text.csv")
# '../data/ml-10M100K/'
data = pd.read_csv('./data/abcnews-date-text.csv', error_bad_lines=False)
print('뉴스 제목 개수 :',len(data))


<ipython-input-8-6b840c013d34>:11: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('./data/abcnews-date-text.csv', error_bad_lines=False)


뉴스 제목 개수 : 1244184


In [9]:
print(data.head(5))

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


In [10]:
text = data[['headline_text']]
text.head(5)

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


2) 텍스트 전처리  
이번 실습에서는 불용어 제거, 표제어 추출, 길이가 짧은 단어 제거라는 세 가지 전처리 기법을 사용합니다.

In [12]:
import nltk
nltk.download('punkt')
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jkm20\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
<ipython-input-12-d9614a4b2542>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)


In [13]:
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])
print(text.head(5))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\jkm20/nltk_data'
    - 'C:\\Users\\jkm20\\anaconda3\\nltk_data'
    - 'C:\\Users\\jkm20\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\jkm20\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\jkm20\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


상위 5개의 샘플에 대해서 불용어를 제거하기 전과 후의 데이터만 비교해도 확실히 몇 가지 단어들이 사라진 것이 보입니다. against, be, of, a, in, to 등의 단어가 제거되었습니다. 이제 표제어 추출을 수행합니다. 표제어 추출로 3인칭 단수 표현을 1인칭으로 바꾸고, 과거 현재형 동사를 현재형으로 바꿉니다.

In [ ]:
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
print(text.head(5))

In [ ]:
tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 3])
print(tokenized_doc[:5])


In [ ]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

# 다시 text['headline_text']에 재저장
text['headline_text'] = detokenized_doc


In [ ]:
text['headline_text'][:5]


In [ ]:
# 상위 1,000개의 단어를 보존 
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000)
X = vectorizer.fit_transform(text['headline_text'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :',X.shape)


In [ ]:
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)


In [ ]:
lda_top = lda_model.fit_transform(X)


In [ ]:
print(lda_model.components_)
print(lda_model.components_.shape) 


In [ ]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)
